In [4]:
conda install swig --yes
!pip install  pyrfr 
!pip install auto-sklearn

SyntaxError: invalid syntax (349373280.py, line 1)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from autosklearn.classification import AutoSklearnClassifier
from autosklearn.metrics import accuracy

# Load data
train_data = pd.read_csv('/Users/laurie/Library/CloudStorage/OneDrive-HKUST(Guangzhou)/hkust-gz/23fall/DSAA6000G/SPACESHIP-TITANIC/data/train.csv')
test_data = pd.read_csv('/Users/laurie/Library/CloudStorage/OneDrive-HKUST(Guangzhou)/hkust-gz/23fall/DSAA6000G/SPACESHIP-TITANIC/data/test.csv')

# Save the target variable and passenger IDs from test set for later use
train_target = train_data['Transported']
test_passenger_ids = test_data['PassengerId']

# Combine train and test data for feature engineering
combined_data = pd.concat([train_data.drop('Transported', axis=1), test_data], ignore_index=True)

# Feature Engineering function
def feature_engineering(data):
    # Handle missing values for numeric columns
    numeric_cols = data.select_dtypes(include=['number']).columns
    data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].mean())

    # Process Cabin column (extracting deck, num, and side)
    data['Deck'] = data['Cabin'].str.split('/', expand=True)[0]
    data['Num'] = data['Cabin'].str.split('/', expand=True)[1]
    data['Side'] = data['Cabin'].str.split('/', expand=True)[2]
    data.drop('Cabin', axis=1, inplace=True)

    # Handle categorical features with one-hot encoding
    data_encoded = pd.get_dummies(data, columns=['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side'], drop_first=True)

    # Convert boolean to numeric
    bool_columns = data_encoded.select_dtypes(include=bool).columns
    data_encoded[bool_columns] = data_encoded[bool_columns].astype(int)

    # Scale numeric features
    scaler = MinMaxScaler()
    data_encoded[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']] = scaler.fit_transform(
        data_encoded[['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']])

    # Drop unnecessary columns
    data_encoded = data_encoded.drop(['PassengerId', 'Name', 'Num'], axis=1)
    
    return data_encoded


combined_data_processed = feature_engineering(combined_data)
train_data_processed = combined_data_processed[:len(train_data)].copy()
test_data_processed = combined_data_processed[len(train_data):].copy()
train_data_processed['Transported'] = train_target

# Model training with hyperparameter optimization
X = train_data_processed.drop('Transported', axis=1)
y = train_data_processed['Transported']

# Splitting the data for training and validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize auto-sklearn classifier
automl = AutoSklearnClassifier(time_left_for_this_task=3600,  # Total time limit for the task
                               per_run_time_limit=300,       # Time limit for a single call to the machine learning model
                               metric=accuracy,              # Performance metric
                               seed=42)                      # Random seed

# Train the model
automl.fit(X_train, y_train)

# Print the final ensemble constructed by auto-sklearn
print(automl.show_models())

# Evaluate the model
y_pred = automl.predict(X_val)
print(f'Accuracy Score: {accuracy_score(y_val, y_pred)}')

# Make predictions on the test set
test_predictions = automl.predict(test_data_processed)

# Prepare submission file
submission = pd.DataFrame({'PassengerId': test_passenger_ids, 'Transported': test_predictions})
submission.to_csv('submission.csv', index=False)

